In [1]:
from PIL import Image
import base64
from openai import OpenAI
from dotenv import load_dotenv
import os
from collections import defaultdict
import os
import random
from tqdm import tqdm

random.seed(42)
load_dotenv()


def resize_image(image_path, size):
    """resize image so that the largest edge is atmost size"""
    img = Image.open(image_path)
    width, height = img.size

    if width <= size and height <= size:
        return img

    if width > height:
        new_width = size
        new_height = int(height * (size / width))
    else:
        new_height = size
        new_width = int(width * (size / height))
    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
    return img


def encode_image(image_path):
    img = resize_image(image_path, 512)
    temp_name = "temp.jpg"
    img.save(temp_name)
    with open(temp_name, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def construct_mcq(options, correct_option):
    correct_option_letter = None
    i = "a"
    mcq = ""

    for option in options:
        if option == correct_option:
            correct_option_letter = i
        mcq += f"{i}. {option}\n"
        i = chr(ord(i) + 1)

    mcq = mcq[:-1]
    return mcq, correct_option_letter


def add_row(content, data, i, with_answer=False):
    content.append(
        {
            "type": "text",
            "text": "Image " + str(i) + ": " + data["question"] + "\n" + data["mcq"],
        }
    )

    encoded_image = encode_image(data["image_path"])

    content.append(
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}",
                "detail": "low",
            },
        }
    )
    if with_answer:
        content.append(
            {
                "type": "text",
                "text": "Answer {}: ".format(i) + data["correct_option_letter"],
            }
        )
    else:
        content.append(
            {
                "type": "text",
                "text": "Answer {}: ".format(i),
            }
        )

    return content

In [2]:
FEWSHOT_JSON = "illusionVQA/comprehension/fewshot_labels.json"
FEWSHOT_IMAGE_DIR = "illusionVQA/comprehension/FEW_SHOTS/"
EVAL_JSON = "illusionVQA/comprehension/eval_labels.json"
EVAL_IMAGE_DIR = "illusionVQA/comprehension/EVAL/"

client = OpenAI(
    base_url="https://openrouter.ai/api/v1", api_key=os.environ["OPENROUTER_API_KEY"]
)
MODEL_NAME = "qwen/qwen-2-vl-72b-instruct"

In [3]:
import json

with open(FEWSHOT_JSON) as f:
    fewshot_dataset = json.load(f)

for data in fewshot_dataset:
    data["image_path"] = FEWSHOT_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(
        data["options"], data["answer"]
    )

In [4]:
with open(EVAL_JSON) as f:
    eval_dataset = json.load(f)

random.shuffle(eval_dataset)

In [5]:
category_count = defaultdict(int)

for data in eval_dataset:
    if data["image"] not in os.listdir(EVAL_IMAGE_DIR):
        print(data["image"])
        continue
    data["image_path"] = EVAL_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(
        data["options"], data["answer"]
    )
    category_count[data["category"]] += 1

print(category_count)
print(len(eval_dataset))

defaultdict(<class 'int'>, {'deceptive design': 37, 'impossible object': 134, 'counting': 11, 'color': 23, 'edited-scene': 21, 'size': 46, 'hidden': 45, 'real-scene': 64, 'angle illusion': 26, 'angle constancy': 2, 'perspective': 2, 'circle-spiral': 6, 'upside-down': 7, 'positive-negative space': 7, 'occlusion': 2, 'repeating pattern': 2})
435


### 0-shot

In [6]:
# content = [
#     {
#         "type": "text",
#         "text": "You'll be given an image, an instruction and some options. You have to select the correct one. Do not explain your reasoning. Answer with only the letter which corresponds to the correct option. Do not repeat the entire answer.",
#     }
# ]
# next_idx = 1

### 4-shot

In [7]:
content = [
    {
        "type": "text",
        "text": "You'll be given an image, an instruction and some choices. You have to select the correct one. Do not explain your reasoning. Answer with the option's letter from the given choices directly. Here are a few examples:",
    }
]


i = 1
for data in fewshot_dataset:
    content = add_row(content, data, i, with_answer=True)
    i += 1

content.append({
                    "type": "text",
                    "text": "Now you try it!",
                })

next_idx = i

### Evaluation Loop

In [8]:
ytrue = []
ypred = []

In [9]:
import time

MAX_RETRIES = 2
for data in tqdm(eval_dataset):
    content_t = add_row(content.copy(), data, next_idx, with_answer=False)
    retries = MAX_RETRIES
    while True:
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {
                        "role": "user",
                        "content": content_t,
                    }
                ],
                max_tokens=5,
            )
            intern_ans = response.choices[0].message.content.strip()[0].lower()
            break
        except Exception as e:
            print(e)
            retries -= 1
            time.sleep(30)
            if retries == 0:
                intern_ans = "GPT4 could not answer this question."
                print("retries exhausted")
                break
            continue

    answer = data["correct_option_letter"].strip()
    ytrue.append(answer)
    ypred.append(intern_ans)

100%|██████████| 435/435 [08:47<00:00,  1.21s/it]


In [10]:
for i in range(len(ypred)):
    if ypred[i] == "GPT4 could not answer this question.":
        print(i)

In [11]:
len(ytrue), len(ypred)

(435, 435)

### Result Analysis

In [12]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

print(accuracy_score(ytrue, ypred))
print(confusion_matrix(ytrue, ypred))
print(classification_report(ytrue, ypred))

print(Counter(ytrue))
print(Counter(ypred))

0.39770114942528734
[[ 0  0  0  0  0  0  0]
 [ 0 44 34 18 16  4  0]
 [ 0 23 44 20 14  8  0]
 [ 0 19 34 42 10  4  0]
 [ 0 13 19 14 38  1  1]
 [ 1  0  0  4  2  4  0]
 [ 0  0  0  1  1  1  1]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           a       0.44      0.38      0.41       116
           b       0.34      0.40      0.37       109
           c       0.42      0.39      0.40       109
           d       0.47      0.44      0.46        86
           e       0.18      0.36      0.24        11
           f       0.50      0.25      0.33         4

    accuracy                           0.40       435
   macro avg       0.34      0.32      0.32       435
weighted avg       0.41      0.40      0.40       435

Counter({'a': 116, 'c': 109, 'b': 109, 'd': 86, 'e': 11, 'f': 4})
Counter({'b': 131, 'a': 99, 'c': 99, 'd': 81, 'e': 22, 'f': 2, '1': 1})


/home/salkhon/miniconda3/envs/molecule/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/salkhon/miniconda3/envs/molecule/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/salkhon/miniconda3/envs/molecule/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [13]:
import prettytable
from collections import defaultdict

table = prettytable.PrettyTable()
table.field_names = ["Category", "Total", "Wrong", "Accuracy"]

got_wrong_dict = defaultdict(int)

for i in range(len(ypred)):
    if ypred[i] != ytrue[i]:
        got_wrong_dict[eval_dataset[i]["category"]] += 1
    else:
        got_wrong_dict[eval_dataset[i]["category"]] += 0


for k, v in got_wrong_dict.items():
    table.add_row([k, category_count[k], v, 1 - (v / category_count[k])])


# sort by total
table.sortby = "Total"
table.reversesort = True
print(table)

+-------------------------+-------+-------+---------------------+
|         Category        | Total | Wrong |       Accuracy      |
+-------------------------+-------+-------+---------------------+
|    impossible object    |  134  |   72  |  0.4626865671641791 |
|        real-scene       |   64  |   37  |       0.421875      |
|           size          |   46  |   30  | 0.34782608695652173 |
|          hidden         |   45  |   38  | 0.15555555555555556 |
|     deceptive design    |   37  |   18  |  0.5135135135135135 |
|      angle illusion     |   26  |   17  | 0.34615384615384615 |
|          color          |   23  |   16  | 0.30434782608695654 |
|       edited-scene      |   21  |   13  | 0.38095238095238093 |
|         counting        |   11  |   9   | 0.18181818181818177 |
|       upside-down       |   7   |   1   |  0.8571428571428572 |
| positive-negative space |   7   |   5   |  0.2857142857142857 |
|      circle-spiral      |   6   |   3   |         0.5         |
|    repea

In [14]:
METRIC_SAVE_DIR = "../../result_jsons/"

eval_dataset_copy = eval_dataset.copy()

print(len(eval_dataset_copy))
for i, data in enumerate(eval_dataset_copy):
    # map letter to option f
    if "BLOCK" in ypred[i]:
        data["vlm_answer"] = "BLOCK"
    else:
        data["vlm_answer"] = ypred[i]

435


In [15]:
import json

with open(METRIC_SAVE_DIR + "qwen2vl_72b_comprehension_4shot.json", "w") as f:
    json.dump(eval_dataset_copy, f, indent=4)